# Download sample `.wav` files and upload to UC Volume

In [0]:
CATALOG = "amine_elhelou"
SCHEMA = "ray_gtm_examples"
VOLUME = "transcribe-data"

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS `{CATALOG}`.`{SCHEMA}`.`{VOLUME}`")

In [0]:
audio_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/raw_audio/LJSpeech"
dbutils.fs.mkdirs(audio_path)

[Download LJSpeech dataset](https://www.kaggle.com/datasets/mathurinache/the-lj-speech-dataset?resource=download) and copy zip folder to your volume

In [0]:
%sh
curl -L -o /tmp/LJSpeech.zip \
  https://www.kaggle.com/api/v1/datasets/download/mathurinache/the-lj-speech-dataset
cp /tmp/LJSpeech.zip /Volumes/amine_elhelou/ray_gtm_examples/transcribe-data/raw_audio/LJSpeech
rm /tmp/LJSpeech.zip

In [0]:
import zipfile


zip_file_path = f"{audio_path}/LJSpeech.zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(audio_path)

In [0]:
import pyspark.sql.functions as F


file_reference_df = spark.createDataFrame(dbutils.fs.ls(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/raw_audio/LJSpeech/LJSpeech-1.1/wavs/"))\
  .withColumn("file_path", F.expr("substring(path, 6, length(path))"))

In [0]:
file_reference_df.display()

In [0]:
file_reference_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{CATALOG}.{SCHEMA}.recordings_file_reference")